In [3]:
import requests
import pandas as pd
import numpy as np
from elasticsearch import Elasticsearch
import os
from datetime import datetime as dt
import os.path
from os import path
import sys
from importlib import reload
import copy
import pickle

In [6]:
sys.path.append('..')

In [7]:
import modules.Elastic_Module
reload(modules.Elastic_Module)

<module 'modules.Elastic_Module' from '/home/francisco/Desktop/repos/opendata/ETL/../modules/Elastic_Module.py'>

In [8]:
from modules.Elastic_Module import *

In [9]:
url_ckan='https://datos.gob.cl/api/3/action/package_list'
url_package='https://datos.gob.cl/api/3/action/package_show?id='
url_package_search='https://datos.gob.cl/api/3/action/package_search?'

In [13]:
#Headers user agent simular browser para no obtener error 403 (prohibido)
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36',
}

In [16]:
#Listas de paquetes ckan (con sus ID)
ckan_packages=requests.get(url_ckan,headers=headers) #consultar api
ckan_packages=ckan_packages.json() #convertir respuesta a json

In [17]:
#OBTENER TODOS LOS PAQUETES DE LA BASE DE DATOS CKAN
rows=str(1000) #maximo numero de resultados por consulta (maximo permitido es 1000)
packs=[] #contenedor de paquetes retornados
for s in range(0,4):
    start=str(s*int(rows)) #paginacion
    packages=requests.get(url_package_search+'rows='+rows+'&'+'start='+start,headers=headers)
    packages=packages.json()
    packs.extend(packages['result']['results'])

In [19]:
recursos=[]
for p in packs:
    for r in p['resources']:
        d=copy.deepcopy(r)
        #parámetros del paquete
        d['package_id']=p['id'] #id
        d['package_name']=p['name'] #nombre
        d['package_title']=p['title'] #titulo
        d['package_notes']=p['notes'] #notas
        if 'organization' in p.keys(): #si es que existe info de organizacion
            if p['organization']!=None:
                d['inst.']=p['organization']['title'] #si es que existe info del titulo incluirlo
                d['inst_name']=p['organization']['name'] #si es que existe info del titulo incluirlo
                d['inst_id']=p['organization']['id'] #si es que existe info del id inst.
            else:
                d['inst.']=None
                d['inst_name']=None
                d['inst_id']=None
        recursos.append(d)
recursos=pd.DataFrame(recursos) #convertir a df      

In [20]:
recursos=recursos.rename(columns={'last_modified':'fecha','format':'formato'}) #cambiar nombres de columnas

In [21]:
recursos=recursos.drop(['mimetype','cache_url','url','cache_last_updated','hash','url_type',
              'mimetype_inner','position','resource_type'],axis=1) #botar columnas

In [22]:
recursos['fecha']=pd.to_datetime(recursos['fecha']) #convertir la fecha a formato solo con la fecha (sin la hora)
#desagregar en dia, mes, año
recursos['dia']=recursos['fecha'].dt.date
recursos['year']=recursos['fecha'].dt.year
recursos['mes']=recursos['fecha'].dt.month

In [24]:
data=recursos.to_dict(orient='records')

In [26]:
#guardar paso en csv
path='../ETLcache/'
now = dt.now()
date_time = now.strftime("_%m-%d-%Y_%H-%M-%S")

In [35]:
recursos.to_csv(path+date_time+'.csv')

In [36]:
#guardar data de paquetes cruda en pickle y borrar para liberar espacio en memoria
with open(path+date_time+'.pickle', 'wb') as f:
    pickle.dump(packs, f)
del(packs)

In [13]:
#es = Elasticsearch(hosts = [ES_HOST],http_auth=('elastic', '                    # replace with the empty string'))

In [38]:
#variables de entorno
args={}
args['elastic_user']='elastic'
args['elastic_psswd']=os.environ['ELASTIC_PASSWD']
args['DOMAIN_ELASTIC']=os.environ['DOMAIN_ELASTIC']
args['ELASTIC_PORT']=os.environ['ELASTIC_PORT']

In [39]:
ES_HOST = "https://"+args['DOMAIN_ELASTIC']
basic_auth={'user':args['elastic_user'], 'psswd':args['elastic_psswd']}

In [40]:
es=elastic_connection(ES_HOST,basic_auth=basic_auth)

In [41]:
es.ping()

True


In [42]:
#generar campos concatenados (parámetros)
generateID_params_custom={'fields':['package_id','id'], #campos a concatenar
                               'head':20, #numero de caracteres a dejar
                                'sep':'_' #separador entre campos
                               }

In [43]:
es.generateID_params_custom=generateID_params_custom

In [44]:
es.add_docs(recursos)

In [45]:
#prepar la data para elastic search
es.elastic_prepare_pattern()

In [46]:
indexes=[
    ('ckan','ckan')
        ]

In [47]:
#generar bulk data
i=indexes[0]
n=1500
id_field='id'
es.bulk_data(INDEX_NAME=i[0],_type=i[1],n=n,id_field=id_field)
#test

creando paquetes de data bulk
size of bulk docs is : 0.1796875 Kbs
N° docs: 10445.0, N° chunks: 14


In [48]:
len(es.chunks[0])

1500

In [49]:
es.clear_index(i[0])

creating 'ckan' index...
 response: '{'acknowledged': True, 'shards_acknowledged': True, 'index': 'ckan'}'


In [50]:
es.bulk_index()

bulk indexing...:index---> ckan
 indexing chunk : 13 of 14finished bulk indexing
No recorded ES indexing errors
No recorded ES indexing exceptions


In [51]:
es.errors

AttributeError: 'elastic_connection' object has no attribute 'errors'

In [55]:
es.test_index(i[0],n=15)

,package_id,datastore_contains_all_records_of_source_file,datastore_active,size,original_url,resource_id,set_url_type,state,ignore_hash,description,...,package_notes,inst.,inst_name,inst_id,document_image_upload,document_clear_upload,document_image_url,dia,year,mes
0,0ee695ff-2340-4cf5-ba50-78b88bfbb5fd,False,False,None,None,None,None,active,None,Archivo de registros de bultos de exportación ...,...,Muestra el registro de ítems correspondientes ...,Servicio Nacional de Aduanas,servicio_nacional_de_aduanas,AE007,None,None,None,2018-06-19,2018.0,6.0
1,0ee695ff-2340-4cf5-ba50-78b88bfbb5fd,False,False,None,None,None,None,active,None,Archivo de registros de ítems correspondientes...,...,Muestra el registro de ítems correspondientes ...,Servicio Nacional de Aduanas,servicio_nacional_de_aduanas,AE007,None,None,None,2018-06-19,2018.0,6.0
2,0ee695ff-2340-4cf5-ba50-78b88bfbb5fd,False,False,None,None,None,None,active,None,Archivo de registros de ítems correspondientes...,...,Muestra el registro de ítems correspondientes ...,Servicio Nacional de Aduanas,servicio_nacional_de_aduanas,AE007,None,None,None,2018-06-19,2018.0,6.0
3,0ee695ff-2340-4cf5-ba50-78b88bfbb5fd,False,False,None,None,None,None,active,None,Archivo de registros de bultos de exportación ...,...,Muestra el registro de ítems correspondientes ...,Servicio Nacional de Aduanas,servicio_nacional_de_aduanas,AE007,None,None,None,2018-06-19,2018.0,6.0
4,0ee695ff-2340-4cf5-ba50-78b88bfbb5fd,False,False,None,None,None,None,active,None,Archivo de registros de ítems correspondientes...,...,Muestra el registro de ítems correspondientes ...,Servicio Nacional de Aduanas,servicio_nacional_de_aduanas,AE007,None,None,None,2018-06-19,2018.0,6.0
5,0ee695ff-2340-4cf5-ba50-78b88bfbb5fd,False,False,None,None,None,None,active,None,Archivo de registros de ítems correspondientes...,...,Muestra el registro de ítems correspondientes ...,Servicio Nacional de Aduanas,servicio_nacional_de_aduanas,AE007,None,None,None,2018-06-19,2018.0,6.0
6,0ee695ff-2340-4cf5-ba50-78b88bfbb5fd,False,False,None,None,None,None,active,None,Archivo de registros de bultos de exportación ...,...,Muestra el registro de ítems correspondientes ...,Servicio Nacional de Aduanas,servicio_nacional_de_aduanas,AE007,None,None,None,2018-06-19,2018.0,6.0
7,0ee695ff-2340-4cf5-ba50-78b88bfbb5fd,False,False,None,None,None,None,active,None,Archivo de registros de ítems correspondientes...,...,Muestra el registro de ítems correspondientes ...,Servicio Nacional de Aduanas,servicio_nacional_de_aduanas,AE007,None,None,None,2018-06-19,2018.0,6.0
8,0ee695ff-2340-4cf5-ba50-78b88bfbb5fd,False,False,None,None,None,None,active,None,Archivo de registros de ítems correspondientes...,...,Muestra el registro de ítems correspondientes ...,Servicio Nacional de Aduanas,servicio_nacional_de_aduanas,AE007,None,None,None,2018-07-26,2018.0,7.0
9,0ee695ff-2340-4cf5-ba50-78b88bfbb5fd,False,False,None,None,None,None,active,None,Archivo de registros de bultos de exportación ...,...,Muestra el registro de ítems correspondientes ...,Servicio Nacional de Aduanas,servicio_nacional_de_aduanas,AE007,None,None,None,2018-06-19,2018.0,6.0


In [53]:
es.list_indices()

['.security-7',
 'kibana_sample_data_flights',
 '.kibana_task_manager',
 '.kibana_1',
 'ckan']